# Clinical Handoff Summary Pipeline

End-to-end pipeline: Load EMR data → Extract findings → Interpret with baseline → Synthesize summary

In [ ]:
import sys
import json
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

from src.config import BASELINE_FILE, INPUT_DATA_FILE, OUTPUT_DIR, EXTRACTIONS_DIR
from src.loader import load_emr_file
from src.extractors import extract_all
from src.interpreter import interpret
from src.synthesizer import synthesize

## 1. Load EMR Excel Files

In [ ]:
baseline_sheets = load_emr_file(BASELINE_FILE)
data_sheets = load_emr_file(INPUT_DATA_FILE)

print(f"Baseline sheets: {list(baseline_sheets.keys())}")
print(f"Data sheets: {list(data_sheets.keys())}")
for name, df in data_sheets.items():
    print(f"  {name}: {len(df)} rows")

## 2. Run Extractors (parallel, Input Data only)

In [ ]:
extractor_outputs = await extract_all(data_sheets)

print(f"Extracted from {len(extractor_outputs)} sheets:")
for eo in extractor_outputs:
    print(f"  {eo.sheet_name}: {len(eo.findings)} findings")

In [ ]:
EXTRACTIONS_DIR.mkdir(parents=True, exist_ok=True)
for eo in extractor_outputs:
    out_path = EXTRACTIONS_DIR / f"{eo.sheet_name.replace(' ', '_').lower()}.json"
    out_path.write_text(json.dumps(eo.model_dump(), ensure_ascii=False, indent=2), encoding="utf-8")
    print(f"Saved: {out_path.name}")

## 3. Run Interpreter (with baseline context)

In [ ]:
interpreter_output = await interpret(extractor_outputs, baseline_sheets)

print(f"Reconciled findings: {len(interpreter_output.reconciled_findings)}")
print(f"Conflicts resolved: {len(interpreter_output.conflicts_resolved)}")
print(f"Duplicates removed: {interpreter_output.duplicates_removed}")

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
interp_path = OUTPUT_DIR / "interpretation.json"
interp_path.write_text(
    json.dumps(interpreter_output.model_dump(), ensure_ascii=False, indent=2),
    encoding="utf-8",
)
print(f"Saved: {interp_path}")

## 4. Run Synthesizer

In [ ]:
synthesizer_output = await synthesize(interpreter_output)

print(f"Findings incorporated: {synthesizer_output.metadata.findings_incorporated}")
print(f"Date range: {synthesizer_output.metadata.date_range}")

In [ ]:
summary_path = OUTPUT_DIR / "summary.md"
summary_path.write_text(synthesizer_output.summary, encoding="utf-8")
print(f"Saved: {summary_path}")

## 5. Final Summary

In [ ]:
from IPython.display import Markdown, display
display(Markdown(synthesizer_output.summary))